<a href="https://colab.research.google.com/github/Hrishx/CNN/blob/main/Cats_and_Dogs_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%matplotlib inline

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

import warnings
warnings.filterwarnings('ignore')

In [8]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

In [9]:
for layer in model_vgg16_conv.layers:
    layer.trainable = False


In [10]:
img_width, img_height = 150, 150
train_data_dir = '/content/drive/MyDrive/data/train'
val_data_dir = '/content/drive/MyDrive/data/validation'
model_weights_file = 'vgg16-xfer-weights.h5'
nb_train_samples = 4
nb_val_samples = 4
nb_epochs = 5

In [11]:

# Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
input = Input(shape=(img_width, img_height, 3))
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)
model = Model(inputs=input, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height),
                                                    batch_size=4, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height),
                                                        batch_size=4,class_mode='categorical')

Found 4 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [13]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit( train_generator, callbacks = callbacks, steps_per_epoch=nb_train_samples,
                              epochs=nb_epochs, validation_data=validation_generator, validation_steps=nb_val_samples)

print('Training Completed!')

Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 873ms/step - accuracy: 0.5000 - loss: 0.7653 - val_accuracy: 0.5000 - val_loss: 4.8651
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 286ms/step - accuracy: 0.5000 - loss: 2.8385 - val_accuracy: 0.5000 - val_loss: 3.4329
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 859ms/step - accuracy: 0.5000 - loss: 0.4241 - val_accuracy: 0.7500 - val_loss: 0.6399
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 291ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 0.7500 - val_loss: 0.6119
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 274ms/step - accuracy: 1.0000 - loss: 0.0114 - val_accuracy: 0.7500 - val_loss: 0.5060
Training Completed!


In [14]:
img_path = '/content/drive/MyDrive/data/cat.jpg'
label = ['Cat','Dog']
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
ind = np.where(features == 1)[1][0]
print('Predicted Array:',features)
print('Predicted Label:',label[ind])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step
Predicted Array: [[1. 0.]]
Predicted Label: Cat
